In [1]:
import zipfile

zip_path = "/content/DataCoSupplyChainDataset.csv.zip"
extract_to = "/content"

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_to)


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

# -----------------------------
# Data Loading and Preprocessing
# -----------------------------
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path, encoding='latin1')
    features = ['order date (DateOrders)', 'Sales', 'Order Item Quantity',
                'Order Item Total', 'Delivery Status', 'Late_delivery_risk']
    df = df[features]
    df['order date (DateOrders)'] = pd.to_datetime(df['order date (DateOrders)'])
    df['day_of_week'] = df['order date (DateOrders)'].dt.dayofweek
    df['month'] = df['order date (DateOrders)'].dt.month
    df['Delivery Status'] = df['Delivery Status'].map({'Late delivery': 0, 'Advance shipping': 1})
    df = df.dropna()
    return df

data = load_and_preprocess_data('DataCoSupplyChainDataset.csv')

In [3]:
# -----------------------------
# Prepare the State Features and Scale
# -----------------------------
state_features = data.drop(['order date (DateOrders)', 'Delivery Status'], axis=1)
scaler = StandardScaler()
scaled_state = scaler.fit_transform(state_features)
scaled_df = pd.DataFrame(scaled_state, columns=state_features.columns)
scaled_df['Delivery Status'] = data['Delivery Status'].values

# Optionally, subsample your data for faster training (e.g., first 1000 samples)
train_subset = scaled_df.iloc[:1000].reset_index(drop=True)

In [14]:
class SupplyChainEnv:
    def __init__(self, data):
        self.data = data
        self.current_step = 0
        # Use the total number of rows as max_steps.
        self.max_steps = len(data)

    def reset(self):
        self.current_step = 0
        return self._get_state()

    def step(self, action):
        # Before calculating reward, check if current_step is out-of-bounds.
        if self.current_step >= self.max_steps:
            return np.zeros(self.state_size()), 0, True

        # Calculate reward for the current step.
        reward = self._calculate_reward(action)
        # Move to the next state.
        self.current_step += 1

        # If after incrementing we are out-of-bounds, return termination.
        if self.current_step >= self.max_steps:
            return np.zeros(self.state_size()), reward, True

        next_state = self._get_state()
        return next_state, reward, False

    def _get_state(self):
        # Guard: if current_step is out-of-bounds, return zeros.
        if self.current_step >= self.max_steps:
            return np.zeros(self.state_size())
        row = self.data.iloc[self.current_step]
        return row.drop('Delivery Status').values

    def _calculate_reward(self, action):
        # Guard: if current_step is out-of-bounds, return neutral reward.
        if self.current_step >= self.max_steps:
            return 0
        actual_status = self.data.iloc[self.current_step]['Delivery Status']
        return 1 if action == actual_status else -1

    def state_size(self):
        return self.data.drop('Delivery Status', axis=1).shape[1]


In [5]:
# -----------------------------
# Define the DQN Agent
# -----------------------------
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self._build_model()

    def _build_model(self):
        model = keras.Sequential([
            keras.Input(shape=(self.state_size,)),
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dense(16, activation='relu'),
            keras.layers.Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=keras.optimizers.Adam(learning_rate=0.001))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return np.random.randint(self.action_size)
        act_values = self.model.predict(state.reshape(1, -1), verbose=0)
        return np.argmax(act_values[0])

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        states = np.array([exp[0] for exp in minibatch])
        actions = np.array([exp[1] for exp in minibatch])
        rewards = np.array([exp[2] for exp in minibatch])
        next_states = np.array([exp[3] for exp in minibatch])
        dones = np.array([exp[4] for exp in minibatch])

        target = self.model.predict(states, verbose=0)
        target_next = self.model.predict(next_states, verbose=0)

        for i in range(batch_size):
            if dones[i]:
                target[i][actions[i]] = rewards[i]
            else:
                target[i][actions[i]] = rewards[i] + self.gamma * np.amax(target_next[i])

        self.model.fit(states, target, epochs=1, verbose=0)

        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

In [19]:
def train_agent(env, agent, episodes, batch_size):
    episode_scores = []  # list to store scores of each episode
    for e in range(episodes):
        state = env.reset()
        total_reward = 0
        while True:
            action = agent.act(state)
            next_state, reward, done = env.step(action)
            agent.remember(state, action, reward, next_state, done)
            state = next_state if not done else np.zeros(env.state_size())
            total_reward += reward
            if done:
                print(f"Episode: {e+1}/{episodes}, Score: {total_reward}")
                episode_scores.append(total_reward)  # record the score
                break
            if len(agent.memory) > batch_size:
                agent.replay(batch_size)
    return episode_scores


In [7]:
# -----------------------------
# Create Environment and Agent
# -----------------------------
env = SupplyChainEnv(train_subset)
state_size = env.state_size()
action_size = 2
agent = DQNAgent(state_size, action_size)

# Train the agent (using fewer episodes and less frequent updates)
train_agent(env, agent, episodes=50, batch_size=32, update_frequency=5)


Episode: 1/50, Score: 297
Episode: 2/50, Score: 747
Episode: 3/50, Score: 903
Episode: 4/50, Score: 945
Episode: 5/50, Score: 979
Episode: 6/50, Score: 995
Episode: 7/50, Score: 995
Episode: 8/50, Score: 991
Episode: 9/50, Score: 987
Episode: 10/50, Score: 991
Episode: 11/50, Score: 987
Episode: 12/50, Score: 983
Episode: 13/50, Score: 985
Episode: 14/50, Score: 993
Episode: 15/50, Score: 985
Episode: 16/50, Score: 989
Episode: 17/50, Score: 991
Episode: 18/50, Score: 987
Episode: 19/50, Score: 983
Episode: 20/50, Score: 987
Episode: 21/50, Score: 995
Episode: 22/50, Score: 985
Episode: 23/50, Score: 987
Episode: 24/50, Score: 989
Episode: 25/50, Score: 991
Episode: 26/50, Score: 989
Episode: 27/50, Score: 993
Episode: 28/50, Score: 991
Episode: 29/50, Score: 997
Episode: 30/50, Score: 989
Episode: 31/50, Score: 989
Episode: 32/50, Score: 995
Episode: 33/50, Score: 987
Episode: 34/50, Score: 981
Episode: 35/50, Score: 989
Episode: 36/50, Score: 993
Episode: 37/50, Score: 989
Episode: 3

In [16]:
# -----------------------------
# Test the Agent on the Last 100 Samples
# -----------------------------
test_data = scaled_df.iloc[-100:].reset_index(drop=True)
env = SupplyChainEnv(test_data)
state = env.reset()
total_reward = 0
done = False

while not done:
    action = agent.act(state)
    next_state, reward, done = env.step(action)
    total_reward += reward
    state = next_state

print(f"Test Score: {total_reward}")


Test Score: 98


In [17]:
agent.model.save("dqn_supply_chain_model.h5")
print("Model saved.")


Model saved.


In [22]:
import pandas as pd
import folium

# Load the raw dataset (using the same CSV file)
data_raw = pd.read_csv('DataCoSupplyChainDataset.csv', encoding='latin1')

# For route visualization, we need geographic coordinates.
# Here, we assume the dataset has 'Latitude' and 'Longitude' columns.
# (Adjust the column names if necessary.)
data_raw = data_raw.dropna(subset=['Latitude', 'Longitude'])
data_raw['Latitude'] = pd.to_numeric(data_raw['Latitude'], errors='coerce')
data_raw['Longitude'] = pd.to_numeric(data_raw['Longitude'], errors='coerce')
data_raw = data_raw.dropna(subset=['Latitude', 'Longitude'])

In [23]:
# For this example, we take unique coordinate pairs (you might need a different logic for your route)
route_points = data_raw[['Latitude', 'Longitude']].drop_duplicates().values.tolist()

# Optionally, if you have an "optimized" order (for example, from a TSP solver) you could use that order.
# For demonstration, we assume the points are already in an order that represents an optimized route.

In [24]:
# Create a Folium map centered on the first coordinate
if route_points:
    m = folium.Map(location=route_points[0], zoom_start=6)

    # Add markers for each point
    for lat, lon in route_points:
        folium.Marker(location=[lat, lon]).add_to(m)

    # Draw a polyline connecting the points
    folium.PolyLine(route_points, color="blue", weight=2.5, opacity=1).add_to(m)

    # Save or display the map (in Jupyter/Colab, displaying m shows the map)
    m.save("optimized_routes.html")
    m  # In a notebook, this displays the map

In [26]:
import folium

# Sample route points (replace with your actual route data)
route_points = [
    [40.7128, -74.0060],   # New York
    [41.8781, -87.6298],   # Chicago
    [34.0522, -118.2437]   # Los Angeles
]

# Create a Folium map centered on the first point
m = folium.Map(location=route_points[0], zoom_start=5)

# Add markers and a polyline connecting the points
for lat, lon in route_points:
    folium.Marker(location=[lat, lon]).add_to(m)

folium.PolyLine(route_points, color="blue", weight=2.5, opacity=1).add_to(m)

m  # Display the map in the notebook


In [29]:
import folium
from geopy.geocoders import Nominatim
import math
import time

# Function to get coordinates for a given city name
def get_coordinates(city):
    geolocator = Nominatim(user_agent="route_optimizer")
    # Nominatim has a rate limit so sleep between calls if needed
    location = geolocator.geocode(city)
    time.sleep(1)  # slight delay to be kind to the server
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Function to compute distance between two lat/lon pairs using the Haversine formula
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Earth's radius in km
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda/2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Get user input: cities separated by commas
cities_input = input("Enter cities separated by commas: ")
cities = [city.strip() for city in cities_input.split(",")]

# Geocode each city to get coordinates
city_coords = {}
for city in cities:
    coord = get_coordinates(city)
    if coord:
        city_coords[city] = coord
    else:
        print(f"Could not find coordinates for: {city}")

# Ensure we have at least two cities
if len(city_coords) < 2:
    print("Need at least two valid cities for route optimization.")
else:
    # List of cities in the order returned (we will reorder them later)
    city_list = list(city_coords.keys())

    # Greedy TSP (nearest neighbor) route optimization
    def greedy_tsp(cities):
        # Start at the first city in the list
        current = cities[0]
        route = [current]
        remaining = set(cities[1:])
        while remaining:
            # Select the city nearest to the current city
            next_city = min(remaining, key=lambda city: haversine(city_coords[current], city_coords[city]))
            route.append(next_city)
            remaining.remove(next_city)
            current = next_city
        return route

    optimal_route = greedy_tsp(city_list)
    print("Optimized route:", optimal_route)

    # Create a Folium map centered on the first city in the optimized route
    m = folium.Map(location=city_coords[optimal_route[0]], zoom_start=5)

    # Add markers for each city and draw the route
    for city in optimal_route:
        folium.Marker(location=city_coords[city], popup=city).add_to(m)

    route_coords = [city_coords[city] for city in optimal_route]
    folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(m)

    # Display the map (in a Jupyter Notebook or Colab, the map will be shown interactively)
    from IPython.display import display
display(m)


Enter cities separated by commas: Dallas, Austin
Optimized route: ['Dallas', 'Austin']


In [30]:
!pip install geopy folium


In [33]:
import folium
from geopy.geocoders import Nominatim
import math
import time

# Function to get coordinates for a given city name
def get_coordinates(city):
    geolocator = Nominatim(user_agent="route_optimizer")
    location = geolocator.geocode(city)
    time.sleep(1)  # delay to respect rate limits
    if location:
        return (location.latitude, location.longitude)
    else:
        return None

# Function to compute the Haversine distance (in miles) between two coordinates
def haversine(coord1, coord2):
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 3958.8  # Radius of Earth in miles
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon2 - lon1)
    a = math.sin(delta_phi / 2.0)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c

# Get user input: cities separated by commas
cities_input = input("Enter cities separated by commas: ")
cities = [city.strip() for city in cities_input.split(",")]

# Geocode each city to get coordinates
city_coords = {}
for city in cities:
    coord = get_coordinates(city)
    if coord:
        city_coords[city] = coord
    else:
        print(f"Could not find coordinates for: {city}")

if len(city_coords) < 2:
    print("Need at least two valid cities for route optimization.")
else:
    # Greedy TSP (nearest neighbor) route optimization
    city_list = list(city_coords.keys())
    def greedy_tsp(cities):
        current = cities[0]
        route = [current]
        remaining = set(cities[1:])
        while remaining:
            next_city = min(remaining, key=lambda city: haversine(city_coords[current], city_coords[city]))
            route.append(next_city)
            remaining.remove(next_city)
            current = next_city
        return route

    optimal_route = greedy_tsp(city_list)
    print("Optimized route:", optimal_route)

    # Build route coordinates list
    route_coords = [city_coords[city] for city in optimal_route]

    # Create a Folium map centered on the first city
    m = folium.Map(location=route_coords[0], zoom_start=5)

    # Add markers for each city with popup labels
    for city in optimal_route:
        folium.Marker(location=city_coords[city], popup=city).add_to(m)

    # Draw polyline for the route
    folium.PolyLine(route_coords, color="blue", weight=2.5, opacity=1).add_to(m)

    # For each segment, calculate distance and travel time, and add an info marker at the midpoint.
    # Assume truck travels 600 miles/day.
    for i in range(len(route_coords) - 1):
        distance = haversine(route_coords[i], route_coords[i+1])
        travel_days = distance / 600  # travel time in days
        # Calculate midpoint
        lat_mid = (route_coords[i][0] + route_coords[i+1][0]) / 2
        lon_mid = (route_coords[i][1] + route_coords[i+1][1]) / 2
        popup_text = f"Distance: {distance:.2f} miles<br>Estimated travel: {travel_days:.2f} days"
        folium.Marker(
            location=[lat_mid, lon_mid],
            popup=popup_text,
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

    # Display the map inline (in Jupyter/Colab) or save it as HTML
    m.save("optimized_route_with_time.html")
    from IPython.display import display
display(m) # In a notebook, this will display the interactive map.


Enter cities separated by commas: Seattle, Dallas
Optimized route: ['Seattle', 'Dallas']
